In [1]:
storage_account_name = "<Storage account name>"
storage_account_key = "<Storage account key>"
container = "<Container name>"

In [2]:
spark.conf.set("fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name), storage_account_key)

In [3]:
data = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("delimiter", ",") \
  .csv("wasbs://{0}@{1}.blob.core.windows.net/housing.csv".format(container, storage_account_name))

data.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252| 452600.0| NEAR BAY|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014| 358500.0| NEAR BAY|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574| 352100.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431| 341300.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462| 342200.0| NEAR BAY|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 5 rows

In [4]:
dbutils.library.installPyPI("mlflow", extras="extras")

Out[ 4 ]: True

In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import mlflow

In [6]:
vectors = VectorAssembler(inputCols= ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income'], outputCol= 'features')

vectors.setParams(handleInvalid="skip")

vector_data = vectors.transform(data)

In [7]:
features = vector_data.select(["features", "median_house_value"])

features.show()

+--------------------+------------------+
 features|median_house_value|
+--------------------+------------------+
[-122.23,37.88,41...| 452600.0|
[-122.22,37.86,21...| 358500.0|
[-122.24,37.85,52...| 352100.0|
[-122.25,37.85,52...| 341300.0|
[-122.25,37.85,52...| 342200.0|
[-122.25,37.85,52...| 269700.0|
[-122.25,37.84,52...| 299200.0|
[-122.25,37.84,52...| 241400.0|
[-122.26,37.84,42...| 226700.0|
[-122.25,37.84,52...| 261100.0|
[-122.26,37.85,52...| 281500.0|
[-122.26,37.85,52...| 241800.0|
[-122.26,37.85,52...| 213500.0|
[-122.26,37.84,52...| 191300.0|
[-122.26,37.85,52...| 159200.0|
[-122.26,37.85,50...| 140000.0|
[-122.27,37.85,52...| 152500.0|
[-122.27,37.85,52...| 155500.0|
[-122.26,37.84,50...| 158700.0|
[-122.27,37.84,52...| 162900.0|
+--------------------+------------------+
only showing top 20 rows

In [8]:
with mlflow.start_run() as run:
  for param in [0.1, 0.00001]:
      lr = LinearRegression(labelCol="median_house_value", featuresCol="features", regParam=param)
      
      model = lr.fit(features)
      
      mlflow.log_metric("R2", model.summary.r2)